<img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" align="right" width="30%"/>

<center>
    <h1>
Dask Tutorial for PyHEP 2022
    </h1>
</center>

Dask is a pure Python library for parallel and distributed computing designed to scale up workflows in the PyData ecosystem.

Dask is has two main components:

- The **collections library(ies)** (sometimes called "Dask core")
  - `dask.array`: chunked NumPy
  - `dask.dataframe`: partitioned Pandas
  - `dask.bag`: partitioned iterables
  - `dask.delayed`: custom algorithms
- The **execution engines** (task schedulers)
  - The distributed engine is its own project (`distributed`, sometimes called "Dask Distributed")

<div style="text-align: center;">
  <img src="https://docs.dask.org/en/stable/_images/dask-overview.svg" align="center" width="70%"/>
</div>

We'll start with a simple `dask.delayed` example that covers _a lot_ of how Dask works:

In [ ]:
from dask.delayed import delayed

def inc(x):
    return x + 1

inc = delayed(inc)

In [ ]:
inc(7)

Notice that this just creates a `Delayed` object.

In [ ]:
delayed_thing = inc(7)

We have to ask Dask to determine the result via `compute()`

In [ ]:
delayed_thing.compute()

We can start to construct a more complex task graph by chaining function calls:

In [ ]:
@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y

In [ ]:
delayed_thing = add(inc(1), inc(2))

In [ ]:
delayed_thing.compute()

We can inspect the complete task graph to see how dask accomplishing computing the result of the collection:

In [ ]:
delayed_task_graph = delayed_thing.dask.to_dict()

In [ ]:
for i, (k, v) in enumerate(delayed_task_graph.items()):
    if i != 0:
        print("\n")
    print("The key (label) of a task:   ", k)
    print("The task itself (Lisp S-exp):", v)

There is a much better method of inspection! (`visualize()`)

In [ ]:
delayed_thing.visualize()

Let's take a look at an example that illustrates something closer to a real workflow: reading and operating on files to produce a histogram. Our example will have two steps:
1. Load an uproot TTree by file and tree name
2. Compute some number from information in the file

In [ ]:
import uproot
import awkward as ak
import hist
import time
from skhep_testdata import data_path

paths = [data_path("uproot-Zmumu.root")] * 5

In [ ]:
@delayed
def read_tree(file_name, tree_name):
    time.sleep(1)  # faking making the file larger
    return uproot.open(file_name)[tree_name]

@delayed
def calculation(tree):
    arrs = tree.arrays()
    h = hist.Hist(hist.axis.Regular(bins=20, start=0, stop=200, name="abs(E1-E2)"))
    h.fill(abs(arrs.E1 - arrs.E2))
    return h

In [ ]:
calculations = []
for p in paths:
    tree = read_tree(p, "events")
    calc = calculation(tree)
    calculations.append(calc)

In [ ]:
calculations

In [ ]:
sum(calculations).visualize()

In [ ]:
sum(calculations).compute()

In [ ]:
%%timeit
sum(calculations).compute()

Let's the same calculation to the non-Dask workflow:

In [ ]:
def read_tree(file_name, tree_name):
    time.sleep(1)  # faking making the file larger
    return uproot.open(file_name)[tree_name]

def calculation(tree):
    arrs = tree.arrays()
    h = hist.Hist(hist.axis.Regular(bins=20, start=0, stop=200, name="abs(E1-E2)"))
    h.fill(abs(arrs.E1 - arrs.E2))
    return h

In [ ]:
%%timeit
calculations = []
for p in paths:
    tree = read_tree(p, "events")
    calc = calculation(tree)
    calculations.append(calc)
sum(calculations)

dask.array
==========

<img src="https://docs.dask.org/en/stable/_images/dask-array.svg" width="50%">
